Install Required Packages

In [ ]:
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126
%pip install matplotlib
%pip install numpy
%pip install tensorboard
%pip install scikit-learn
%pip install seaborn

In [ ]:
# Fix Tensorboard Ghost Processes
import tempfile
import shutil
import os
shutil.rmtree(os.path.join(tempfile.gettempdir(), '.tensorboard-info'))

In [ ]:
# Set Up
import os
import torch

import torchvision

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import numpy as np

from DataLoader import DataLoader
from MLP import MLP
from VGG16 import VGG16
from PreTrainedVGG16 import PreTrainedVGG16
from Trainer import Trainer

def img_show(img):
    img = img / 2 + 0.5 # unnormalize
    np_img = img.cpu().numpy()
    plt.imshow(np.transpose(np_img, (1, 2, 0)))
    plt.show()

display_labels = ["accessories", "jackets", "jeans", "knitwear", "shirts", "shoes", "shorts", "tees"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Setup MLP model for Training

In [ ]:
# Train Model
%load_ext tensorboard
train_set = DataLoader(data_dir=os.getcwd(), trans_width=124, trans_height=124).load(dataset="/data/train", batch_size=32, shuffle=True, workers=2)
valid_loader = DataLoader(data_dir=os.getcwd(), trans_width=124, trans_height=124).load(dataset="/data/valid", batch_size=10, shuffle=False, workers=2)
mlp_model = MLP(log_dir=os.getcwd()+"/MLP", lr=1e-04).to(device)
mlp_trainer = Trainer(log_dir=os.getcwd(), n_epochs=6, device=device)

Train MLP Model

In [ ]:
%load_ext tensorboard
mlp_trainer.fit(mlp_model, train_set, valid_loader)

In [ ]:
plt.clf()
plt.plot(getattr(mlp_trainer, "avg_train_loss"), label="Avg. Training Loss")
plt.plot(getattr(mlp_trainer, "avg_valid_loss"), label="Avg. Validation Loss")
plt.plot(getattr(mlp_trainer, "training_accuracy"), label="Training Accuracy (x100)%")
plt.plot(getattr(mlp_trainer, "validation_accuracy"), label="Validation Accuracy (x100)%")
plt.legend()
plt.savefig(os.getcwd() + "/MLP/MLP_Loss_Plot.jpg")

Write MLP Loss to Tensorboard

In [ ]:
%load_ext tensorboard
for epoch in range(len(mlp_trainer.avg_train_loss)):
    mlp_model.writer.add_scalars('MLP', {'Avg_Training_Loss': mlp_trainer.avg_train_loss[epoch],
                                         'Avg_Validation_Loss': mlp_trainer.avg_valid_loss[epoch],
                                         'Training_Accuracy(x100)%': mlp_trainer.training_accuracy[epoch],
                                         'Validation_Accuracy(x100)%': mlp_trainer.validation_accuracy[epoch]}, epoch)

In [ ]:
mlp_model.writer.close()

In [ ]:
%tensorboard --logdir=./MLP

Test MLP Model

In [ ]:
test_loader = DataLoader(data_dir=os.getcwd(), trans_width=124, trans_height=124).load(dataset="/data/test", batch_size=10, shuffle=False, workers=2)
classes = train_set.dataset.classes
test_ground_truth = [4, 4, 2, 2, 7, 3, 5]
dataiter = iter(test_loader)
images, labels = next(dataiter)

images = images.to(device)

img_show(torchvision.utils.make_grid(images))
mlp_model.eval()
output = mlp_model(images).to(device)
estimatedLabels = torch.max(output, 1).indices

print('Estimated Labels: ', ' '.join(f'{classes[estimatedLabels[j]]:5s}' for j in range(images.shape[0])))

print(estimatedLabels, test_ground_truth)

cm = confusion_matrix(y_pred=estimatedLabels.cpu(), y_true=test_ground_truth, labels=display_labels)
dcm = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)
dcm.plot()
plt.show()
plt.savefig(os.getcwd() + "/MLP/MLP_Confusion_Matrix.jpg")

In [ ]:
# resume = False
# if resume:
#   epochs = model.load(dir="Mydrive/ICT303_Results/MLP/MLP_Epoch_11.tar")
#   trainer.load(dir=f"Mydrive/ICT303_Results/MLP/MLP_Epoch_{epochs}_TrainLoss.tar")
#   # validator.load(dir=f"Mydrive/ICT303_Results/MLP/MLP_Epoch_{epochs}_ValLoss.tar")

Setup VGG16 Model

In [ ]:
# Train VGG16 Model
%load_ext tensorboard
train_loader = DataLoader(data_dir=os.getcwd(), trans_width=224, trans_height=224).load(dataset="/data/train", batch_size=32, shuffle=True, workers=2)
valid_loader = DataLoader(data_dir=os.getcwd(), trans_width=224, trans_height=224).load(dataset="/data/valid", batch_size=10, shuffle=False, workers=2)
vgg16_model = VGG16(log_dir=os.getcwd()+"/VGG16", output_size=8, lr=1e-4).to(device)
vgg16_trainer = Trainer(log_dir=os.getcwd()+"/VGG16", n_epochs=10, device=device, use_lr_scheduler=True)

In [ ]:
%load_ext tensorboard
vgg16_trainer.fit(vgg16_model, train_loader, valid_loader)

In [ ]:
plt.clf()
plt.plot(getattr(vgg16_trainer, "avg_train_loss"), label="Avg. Training Loss")
plt.plot(getattr(vgg16_trainer, "avg_valid_loss"), label="Avg. Validation Loss")
plt.plot(getattr(vgg16_trainer, "training_accuracy"), label="Training Accuracy (x100)%")
plt.plot(getattr(vgg16_trainer, "validation_accuracy"), label="Validation Accuracy (x100)%")
plt.legend()
plt.savefig(os.getcwd() + "/VGG16/VGG16_Loss_Plot.jpg")

In [ ]:
%load_ext tensorboard
for epoch in range(len(vgg16_trainer.avg_train_loss)):
    vgg16_model.writer.add_scalars('VGG16', {'Avg_Training_Loss': vgg16_trainer.avg_train_loss[epoch],
                                             'Avg_Validation_Loss': vgg16_trainer.avg_valid_loss[epoch],
                                             'Training_Accuracy(x100)%': vgg16_trainer.training_accuracy[epoch],
                                             'Validation_Accuracy(x100)%': vgg16_trainer.validation_accuracy[epoch]}, epoch)

In [ ]:
vgg16_model.writer.close()

In [ ]:
%tensorboard --logdir=./VGG16

In [ ]:
test_loader = DataLoader(data_dir=os.getcwd(), trans_width=224, trans_height=224).load(dataset="/data/test", batch_size=10, shuffle=False, workers=2)
classes = train_loader.dataset.classes
dataiter = iter(test_loader)
images, labels = next(dataiter)

images = images.to(device)

img_show(torchvision.utils.make_grid(images))
vgg16_model.eval()
output = vgg16_model(images).to(device)
estimatedLabels = torch.max(output, 1).indices

print('Estimated Labels: ', ' '.join(f'{classes[estimatedLabels[j]]:5s}' for j in range(images.shape[0])))

cm = confusion_matrix(estimatedLabels.cpu(), test_ground_truth)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(8), yticklabels=range(8))
plt.ylabel("Ground Truth")
plt.xlabel("Predicted")
plt.title("Confusion Matrix")
plt.show()
plt.savefig(os.getcwd() + "/VGG16/VGG16_Confusion_Matrix.jpg")

Setup PreTrained VGG16 Model for Training (Just last layer of classifier)

In [ ]:
%load_ext tensorboard
train_set = DataLoader(data_dir=os.getcwd(), trans_width=224, trans_height=224).load(dataset="/data/train", batch_size=32, shuffle=True, workers=2)
valid_loader = DataLoader(data_dir=os.getcwd(), trans_width=224, trans_height=224).load(dataset="/data/valid", batch_size=10, shuffle=False, workers=2)
torch_vgg16_model = PreTrainedVGG16(log_dir=os.getcwd()+"/PreTrainedVGG16", lr=0.001).to(device)
torch_vgg16_trainer = Trainer(log_dir=os.getcwd(), n_epochs=3, device=device)

Train PreTrained VGG16 Model

In [ ]:
%load_ext tensorboard
torch_vgg16_trainer.fit(torch_vgg16_model, train_set, valid_loader)

In [ ]:
plt.clf()
plt.plot(getattr(torch_vgg16_trainer, "avg_train_loss"), label="Avg. Training Loss")
plt.plot(getattr(torch_vgg16_trainer, "avg_valid_loss"), label="Avg. Validation Loss")
plt.plot(getattr(torch_vgg16_trainer, "training_accuracy"), label="Training Accuracy (%)")
plt.plot(getattr(torch_vgg16_trainer, "validation_accuracy"), label="Validation Accuracy (%)")
plt.legend()
plt.savefig(os.getcwd() + "/VGG16/VGG16_Loss_Plot.jpg")

Test PreTrained VGG16 Model

In [ ]:
%load_ext tensorboard
for epoch in range(len(torch_vgg16_trainer.avg_train_loss)):
    torch_vgg16_model.writer.add_scalars('PreTrainedVGG16', {'Avg_Training_Loss': torch_vgg16_trainer.avg_train_loss[epoch],
                                                             'Avg_Validation_Loss': torch_vgg16_trainer.avg_valid_loss[epoch],
                                                             'Training_Accuracy(x100)%': torch_vgg16_trainer.training_accuracy[epoch],
                                                             'Validation_Accuracy(x100)%': torch_vgg16_trainer.validation_accuracy[epoch]}, epoch)

In [ ]:
torch_vgg16_model.writer.close()

In [ ]:
%tensorboard --logdir=./PreTrainedVGG16

In [ ]:
test_loader = DataLoader(data_dir=os.getcwd(), trans_width=224, trans_height=224).load(dataset="/data/test", batch_size=10, shuffle=False, workers=2)
classes = train_set.dataset.classes
dataiter = iter(test_loader)
images, labels = next(dataiter)

images = images.to(device)

img_show(torchvision.utils.make_grid(images))
torch_vgg16_model.eval()
output = torch_vgg16_model(images).to(device)
estimatedLabels = torch.max(output, 1).indices

print('Estimated Labels: ', ' '.join(f'{classes[estimatedLabels[j]]:5s}' for j in range(images.shape[0])))

cm = confusion_matrix(estimatedLabels.cpu(), test_ground_truth)
print(estimatedLabels, test_ground_truth)
sns.heatmap(cm, annot=True, xticklabels=range(8), yticklabels=range(8))
plt.ylabel("Ground Truth")
plt.xlabel("Predicted")
plt.title("Confusion Matrix")
plt.show()
plt.savefig(os.getcwd() + "/PreTrainedVGG16/PreTrainedVGG16_Confusion_Matrix.jpg")